## Import

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import re
from nltk import tokenize

# API Header

Entity Search API available @ PermID.org
url is for production version

In [3]:
url = 'http://api.apigarden-qa.int.thomsonreuters.com/permid/search?q='

#url = 'https://api.thomsonreuters.com/permid/search?q='
access_token = 'YOUR_ACCESS_TOKEN'
headers = {'X-AG-Access-Token' : access_token}

# Extracts all mentions previously identified by Calais API code.

In [2]:
def allentities(input_permid):
    table = tagged_df[tagged_df.permid == input_permid].reset_index(drop=True)
    elist = []
    for index, row in table.iterrows():
        elist = elist + row['entities']
    elist = list(set(elist))
    return elist

# Extracts the name in PermID database, and possible abbreviation (that is usually identifiable in url)

Bank of America may have url: "www.boa.com" and boa may be used as abbreviation

In [4]:
def returnEntity(permid):
    input_url = url + str(permid)
    response = requests.get(input_url, headers=headers)
  
    try:
        response = requests.get(input_url, headers=headers)
        print 'status code: %s' % response.status_code
        if (response.status_code == 429) | (response.status_code == 500):
            print "Retrying..."
            time.sleep(1.0)
            return returnEntity(permid)
        else:
            contents = json.loads(response.content)['result']['organizations']['entities'][0]
            bankname = contents['organizationName']
            if contents.has_key('hasURL'):
                output_url = contents['hasURL'].lstrip('http://').lstrip('https://').lstrip('www.').split('.')[0]
                return bankname, output_url
            else:
                return bankname, ''
        
    except:
        print "Error 413"
        return(json.loads('{}'))

# All possible variation of entity names

"XYZ Bank Ltd" -> "XYZ Bank"

In [1]:
def banknamecomb(permid):
    name, url_name = returnEntity(permid)
    wordlist = name.split()
    previous_use = allentities(permid)
    
    bank_var = ['Bank','Banca','Banco']
    suffix = ['A/S','SpA','SA','PLC','Ltd','AG','NV','ASA','Bhd','SE','Inc','SAU']
    
    name_list = [name]
    name_list = name_list + previous_use
    
    if url_name != '':
        name_list.append(url_name)
        
    for y in name_list:
        y_words = y.split()
        if (y_words[-1][0] == '(') & (y_words[-1][-1] == ')'):
            name_list.append(y_words[-1].lstrip('(').rstrip(')'))
            
    for i in bank_var:
        if (i in wordlist):
            if wordlist.index(i) != 0:
                x = wordlist.index(i)
                name_list.append(wordlist[x-1] + wordlist[x])
                
    for j in suffix:
        name_list.append(name.rstrip(j))
        for k in previous_use:
            name_list.append(k.rstrip(j))   
            
    for q in bank_var:
        for s in wordlist:
            if (q != s) & (q.lower() in s.lower()):
                name_list.append(s)
    
    ## Some obvious exceptions.
    ## Alpha was removed, because "Alpha" tends to cause Newsroom to collect news articles discussing statistical analysis
    if url_name == 'alpha':
        try:
            name_list.remove('alpha')
        except:
            pass
    
    if url_name == 'tb':
        try:
            name_list.remove('tb')
        except:
            pass
        
    if url_name == 'ortal':
        try:
            name_list.remove('ortal')
        except:
            pass
        
    name_list = list(set(name_list))
    name_list = [i for i in name_list if len(i) > 2]
    return name_list

Identify all cases where we have period followed by no space.

In [105]:
def clean_paragraph(input_para):
    ## Only loophole is when we have a sentence ending with a number and succeeding senetence begins with a number.
    ## Ex: "Michael is 23. 10 students are younger than Mike."
    possible_exceptions = set(re.findall('([0-9A-Za-z]+[.][0-9A-Za-z]+)',input_para))
    allowed_list = set(re.findall('\d[.]\d',input_para))
    
    exception_list = list(possible_exceptions.difference(allowed_list))
    
    new_list = [e.replace('.','. ') for e in exception_list]
    
    replace_dict = dict(zip(exception_list, new_list))
    
    new_para = input_para
    
    for old,new in replace_dict.items():
        new_para = new_para.replace(old,new)
        new_para = new_para.replace(';',',') ## Using ; as separator, so replacing any ; with ,
        
    return new_para

In [107]:
for index, row in empty_df.iterrows():
    
    print index
    
    permid = row['permid']
    
    if permid == 4295870630:
        continue

    txt = str(row['txt'])
    
    clean_para = clean_paragraph(txt)
    sent_list= tokenize.sent_tokenize(clean_para.decode('utf-8'))
    clean_sent_list = [sent.strip().encode('utf-8') for sent in sent_list]
    
    entity_candidate = banknamecomb(permid)
    
    entity_list = []
    
    for e in entity_candidate:
        if str(e) in str(txt):
            entity_list.append(str(e))
            
    entity_list = list(set(entity_list))

    empty_df['entities'][index] = entity_list
    empty_df['sentences'][index] = [sent for sent in clean_sent_list 
                                if any(entity.lower() in sent.lower() for entity in entity_list)]

        

1357
status code: 200
1358


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


status code: 200
1359
status code: 200
1360
status code: 200
1361
status code: 200
1362
status code: 200
1363
status code: 200
1364
status code: 200
1365
status code: 200
1366
status code: 200
1367
status code: 200
1368
status code: 200
1369
status code: 200
1370
status code: 200
1371
status code: 200
1372
status code: 200
1373
status code: 200
1374
status code: 200
1375
status code: 200
1376
status code: 200
1377
status code: 200
1378
status code: 200
1379
status code: 200
1380
status code: 200
1381
status code: 200
1382
status code: 200
1383
status code: 200
1384
status code: 200
1385
status code: 200
1386
status code: 200
1387
status code: 200
1388
status code: 200
1389
status code: 200
1390
status code: 200
1391
status code: 200
1392
status code: 200
1393
status code: 200
1394
status code: 200
1395
status code: 200
1396
status code: 200
1397
status code: 200
1398
status code: 200
1399
status code: 200
1400
status code: 200
1401
status code: 200
1402
status code: 200
1403
status cod